In [1]:
!git clone https://github.com/nateraw/huggingface-vit-finetune.git
%cd huggingface-vit-finetune/
!pip install -r requirements.txt
!pip install git+https://github.com/huggingface/transformers.git@master --upgrade
!pip install torchattacks

Cloning into 'huggingface-vit-finetune'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 24 (delta 10), reused 19 (delta 6), pack-reused 0
Unpacking objects: 100% (24/24), done.
/content/huggingface-vit-finetune
     |████████████████████████████████| 849kB 9.0MB/s 
     |████████████████████████████████| 829kB 25.9MB/s 
     |████████████████████████████████| 276kB 47.0MB/s 
     |████████████████████████████████| 112kB 46.6MB/s 
     |████████████████████████████████| 276kB 49.8MB/s 
     |████████████████████████████████| 1.3MB 52.5MB/s 
     |████████████████████████████████| 143kB 34.6MB/s 
     |████████████████████████████████| 296kB 55.7MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=7e95a3ce3cd4f94d7955f2de0358af9329d423b220b9f18a1bc17bbb4a70851d
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from transformers import ViTFeatureExtractor, ViTForImageClassification
import requests

import torchattacks
import time
import json

In [3]:
result_dict = {}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
transform = transforms.Compose(
    [
      # transforms.Resize((224, 224)),
      transforms.ToTensor(),
      # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
cifar_trainloader = torch.utils.data.DataLoader(cifar_trainset, batch_size=64, shuffle=True, num_workers=2)
cifar_testloader = torch.utils.data.DataLoader(cifar_testset, batch_size=64, shuffle=False, num_workers=2)
cifar_classes = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'
]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
class Transformer(nn.Module):
  def __init__(self, model):
    super(Transformer, self).__init__()
    self.model = model
  def forward(self, x):
    return self.model(x)[0]


In [7]:
model = ViTForImageClassification.from_pretrained('nateraw/vit-base-patch16-224-cifar10', return_dict=False)
model.to(device)
model = nn.Sequential(
    nn.Upsample(scale_factor=7, mode='bilinear'),
    model
)
ViT_model = Transformer(model).to(device)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(ViT_model.parameters(), lr=0.0002)

In [ ]:
# cifar-10 DATASET consists of 60000 images
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["Transformer"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 test images: 17.187500 %
Accuracy of the network on 704 cifar-10 test images: 11.789773 %
Accuracy of the network on 1344 cifar-10 test images: 11.458333 %
Accuracy of the network on 1984 cifar-10 test images: 10.735887 %
Accuracy of the network on 2624 cifar-10 test images: 10.556402 %
Accuracy of the network on 3264 cifar-10 test images: 10.079657 %
Accuracy of the network on 3904 cifar-10 test images: 9.912910 %
Accuracy of the network on 4544 cifar-10 test images: 9.727113 %
Accuracy of the network on 5184 cifar-10 test images: 9.934414 %
Accuracy of the network on 5824 cifar-10 test images: 10.027473 %
Accuracy of the network on 6464 cifar-10 test images: 9.916460 %
Accuracy of the network on 7104 cifar-10 test images: 9.952140 %
Accuracy of the network on 7744 cifar-10 test images: 9.865702 %
Accuracy of the network on 8384 cifar-10 test images: 9.887882 %
Accuracy of the network on 9024 cifar-10 test images: 10.095301 %
Accuracy of the netw

In [ ]:
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_trainloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 10 == 0):
      print('Accuracy of the network on %d cifar-10 train images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 train images: %f %%' % (
    total,
    100 * correct / total))
result_dict["Transformer"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 train images: 100.000000 %
Accuracy of the network on 704 cifar-10 train images: 100.000000 %
Accuracy of the network on 1344 cifar-10 train images: 100.000000 %
Accuracy of the network on 1984 cifar-10 train images: 100.000000 %
Accuracy of the network on 2624 cifar-10 train images: 100.000000 %
Accuracy of the network on 3264 cifar-10 train images: 100.000000 %
Accuracy of the network on 3904 cifar-10 train images: 100.000000 %
Accuracy of the network on 4544 cifar-10 train images: 99.977993 %
Accuracy of the network on 5184 cifar-10 train images: 99.980710 %
Accuracy of the network on 5824 cifar-10 train images: 99.965659 %
Accuracy of the network on 6464 cifar-10 train images: 99.969059 %
Accuracy of the network on 7104 cifar-10 train images: 99.971847 %
Accuracy of the network on 7744 cifar-10 train images: 99.974174 %
Accuracy of the network on 8384 cifar-10 train images: 99.976145 %
Accuracy of the network on 9024 cifar-10 train images: 99.

KeyboardInterrupt: ignored

In [10]:
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_trainloader:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  if i == 0:
    print(predicted, labels, total)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d clean train data: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d clean cifar-10 train images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ep1_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ep1_train_sec"] = t1 - t0
result_dict

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


tensor([7, 2, 0, 4, 0, 8, 6, 8, 0, 0, 5, 7, 0, 8, 1, 3, 4, 5, 3, 0, 4, 0, 1, 0,
        4, 1, 2, 6, 8, 3, 6, 8, 0, 1, 1, 8, 3, 2, 5, 4, 7, 7, 7, 3, 5, 2, 5, 0,
        7, 4, 3, 6, 8, 1, 5, 8, 7, 8, 1, 0, 0, 0, 0, 5], device='cuda:0') tensor([7, 2, 0, 4, 0, 8, 6, 8, 9, 0, 5, 7, 8, 8, 9, 3, 4, 5, 3, 9, 4, 9, 1, 9,
        4, 1, 2, 6, 8, 3, 6, 8, 0, 1, 1, 8, 3, 2, 5, 4, 7, 7, 7, 3, 5, 2, 5, 9,
        7, 4, 3, 6, 8, 1, 5, 8, 7, 8, 1, 0, 9, 0, 8, 5], device='cuda:0') 64
Training accuracy of ViT on 64 clean train data: 85.937500 %
running loss:  0.005255349911749363
Training accuracy of ViT on 9664 clean train data: 98.727235 %
running loss:  22.896505969576538


KeyboardInterrupt: ignored

# ATTA Prepare Evaluation Adversarial Examples


In [9]:
atk_pgd_5 = torchattacks.PGD(ViT_model, eps=8/255, alpha=2/255, steps=5)
atk_pgd_10 = torchattacks.PGD(ViT_model, eps=8/255, alpha=2/255, steps=10)
atk_pgd_20 = torchattacks.PGD(ViT_model, eps=8/255, alpha=2/255, steps=20)

In [10]:
cifar_adv_test10 = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_testloader):
    adv_images = atk_pgd_10(inputs, labels)
    cifar_adv_test10.append((adv_images.cpu(), labels.cpu()))
    if i % 100 == 0:
      print("Progress(%): ", i*64/100)
      
print("total time(sec) : %.2f" % (time.time() - start))

# Store pgd 10 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd10_8p_adv_test.pt'
!touch $outfile_name
torch.save(cifar_adv_test10, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  64.0
total time(sec) : 979.84
ATTA_5_adv_itr1_model.pt  pgd_5_adv_itr1.pt	      pgd_5_adv_itr4_model.pt
ATTA_5_adv_itr2_model.pt  pgd_5_adv_itr1_v2_model.pt  pgd_5_adv_itr4.pt
pgd10_8p_adv_test.pt	  pgd_5_adv_itr1_v2.pt	      pgd_5_adv_itr5_model.pt
pgd_10_adv_test.pt	  pgd_5_adv_itr2_model.pt     pgd_5_adv_itr5.pt
pgd_20_adv_test.pt	  pgd_5_adv_itr2.pt	      README.gdoc
pgd_5_adv_itr1.json	  pgd_5_adv_itr3_model.pt
pgd_5_adv_itr1_model.pt   pgd_5_adv_itr3.pt


In [11]:
# Load pgd 10 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd10_8p_adv_test.pt'
cifar_adv_test10 = torch.load(outfile_name)
len(cifar_adv_test10)

157

In [11]:
cifar_adv_test20 = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_testloader):
    adv_images = atk_pgd_20(inputs, labels)
    cifar_adv_test20.append((adv_images.cpu(), labels.cpu()))
    if i % 100 == 0:
      print("Progress(%): ", i*64/100)

print("total time(sec) : %.2f" % (time.time() - start))

# Store pgd 20 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd20_8p_adv_test.pt'
!touch $outfile_name
torch.save(cifar_adv_test20, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  64.0
total time(sec) : 1962.12
ATTA_5_adv_itr1_model.pt  pgd_5_adv_itr1_model.pt     pgd_5_adv_itr3.pt
ATTA_5_adv_itr2_model.pt  pgd_5_adv_itr1.pt	      pgd_5_adv_itr4_model.pt
pgd10_8p_adv_test.pt	  pgd_5_adv_itr1_v2_model.pt  pgd_5_adv_itr4.pt
pgd_10_adv_test.pt	  pgd_5_adv_itr1_v2.pt	      pgd_5_adv_itr5_model.pt
pgd20_8p_adv_test.pt	  pgd_5_adv_itr2_model.pt     pgd_5_adv_itr5.pt
pgd_20_adv_test.pt	  pgd_5_adv_itr2.pt	      README.gdoc
pgd_5_adv_itr1.json	  pgd_5_adv_itr3_model.pt


In [12]:
# Load pgd 20 attacks
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd20_8p_adv_test.pt'
cifar_adv_test20 = torch.load(outfile_name)
len(cifar_adv_test20)

157

# ATTA Training Epoch 1

In [12]:
# Generate first itr adv images
cifar_adv = []
start = time.time()
for i, (inputs, labels) in enumerate(cifar_trainloader):
    adv_images = atk_pgd_5(inputs, labels)
    cifar_adv.append((adv_images.cpu(), labels.cpu()))
    if i % 100 == 0:
      print("Progress(%): ", 64*i/500)

print("total time(sec) : %.2f" % (time.time() - start))

# Store
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd5_8p_adv_itr1.pt'
!touch $outfile_name
torch.save(cifar_adv, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  12.8
Progress(%):  25.6
Progress(%):  38.4
Progress(%):  51.2
Progress(%):  64.0
Progress(%):  76.8
Progress(%):  89.6
total time(sec) : 2458.99
ATTA_5_adv_itr1_model.pt  pgd_5_adv_itr1.json	      pgd_5_adv_itr3_model.pt
ATTA_5_adv_itr2_model.pt  pgd_5_adv_itr1_model.pt     pgd_5_adv_itr3.pt
pgd10_8p_adv_test.pt	  pgd_5_adv_itr1.pt	      pgd_5_adv_itr4_model.pt
pgd_10_adv_test.pt	  pgd_5_adv_itr1_v2_model.pt  pgd_5_adv_itr4.pt
pgd20_8p_adv_test.pt	  pgd_5_adv_itr1_v2.pt	      pgd_5_adv_itr5_model.pt
pgd_20_adv_test.pt	  pgd_5_adv_itr2_model.pt     pgd_5_adv_itr5.pt
pgd5_8p_adv_itr1.pt	  pgd_5_adv_itr2.pt	      README.gdoc


In [10]:
# Load
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd5_8p_adv_itr1.pt'
cifar_adv = torch.load(outfile_name)
len(cifar_adv)

782

In [13]:
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep1_train_sec"] = t1 - t0
result_dict

# Store model after itr1
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr1_model.pt'
!touch $outfile_name
torch.save(ViT_model, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 25.000000 %
running loss:  6.625616550445557
Training accuracy of ViT on 9664 adv exs: 86.961921 %
running loss:  104.10059168934822
Training accuracy of ViT on 19264 adv exs: 90.297965 %
running loss:  161.9505341053009
Training accuracy of ViT on 28864 adv exs: 91.868764 %
running loss:  211.75836592912674
Training accuracy of ViT on 38464 adv exs: 92.853057 %
running loss:  256.01912017166615
Training accuracy of ViT on 48064 adv exs: 93.652214 %
running loss:  293.8845605030656
Training accuracy on 50000 pgd-5 cifar-10 adv images: 93.790000 %
ATTA5_8p_adv_itr1_model.pt  pgd_5_adv_itr1.json		pgd_5_adv_itr3.pt
ATTA_5_adv_itr1_model.pt    pgd_5_adv_itr1_model.pt	pgd_5_adv_itr4_model.pt
ATTA_5_adv_itr2_model.pt    pgd_5_adv_itr1.pt		pgd_5_adv_itr4.pt
pgd10_8p_adv_test.pt	    pgd_5_adv_itr1_v2_model.pt	pgd_5_adv_itr5_model.pt
pgd_10_adv_test.pt	    pgd_5_adv_itr1_v2.pt	pgd_5_adv_itr5.pt
pgd20_8p_adv_test.pt	    pgd_5_adv_itr2_model.pt	README.gdoc


In [7]:
# Load model
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr1_model.pt'
ViT_model = torch.load(outfile_name)
ViT_model.to(device)

Transformer(
  (model): Sequential(
    (0): Upsample(scale_factor=7.0, mode=bilinear)
    (1): ViTForImageClassification(
      (vit): ViTModel(
        (embeddings): ViTEmbeddings(
          (patch_embeddings): PatchEmbeddings(
            (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          )
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (encoder): ViTEncoder(
          (layer): ModuleList(
            (0): ViTLayer(
              (attention): ViTAttention(
                (attention): ViTSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): ViTSelfOutput(
                  (dense): Linear(in_features=768, out_features=768, bias=True)
    

In [14]:
result_dict

{'ViT_ATTA_5_ep1_train_acc': 93.79,
 'ViT_ATTA_5_ep1_train_sec': 663.3380136489868}

# Evaluate ATTA Epoch 1 Model

In [16]:
# Epoch 1 Model on Original test data
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 50 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ATTA_ep1_Orig_Acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 test images: 93.750000 %
Accuracy of the network on 704 cifar-10 test images: 92.897727 %
Accuracy of the network on 1344 cifar-10 test images: 93.154762 %
Accuracy of the network on 1984 cifar-10 test images: 92.691532 %
Accuracy of the network on 2624 cifar-10 test images: 92.530488 %
Accuracy of the network on 3264 cifar-10 test images: 93.075980 %
Accuracy of the network on 3904 cifar-10 test images: 92.776639 %
Accuracy of the network on 4544 cifar-10 test images: 93.001761 %
Accuracy of the network on 5184 cifar-10 test images: 93.055556 %
Accuracy of the network on 5824 cifar-10 test images: 93.131868 %
Accuracy of the network on 6464 cifar-10 test images: 93.177599 %
Accuracy of the network on 7104 cifar-10 test images: 93.172860 %
Accuracy of the network on 7744 cifar-10 test images: 93.168905 %
Accuracy of the network on 8384 cifar-10 test images: 93.093989 %
Accuracy of the network on 9024 cifar-10 test images: 93.151596 %
Accuracy of t

In [15]:
# Evaluate model on pgd-10 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test10):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep1 ViT on %d pgd10 cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_test_pgd10_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 87.500000 %
Accuracy on 6464 cifar-10 test images: 91.321163 %
Accuracy ATTA5-ep1 ViT on 10000 cifar-10 test images: 91.290000 %


In [17]:
# Evaluate model on pgd-20 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test20):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep1 ViT on %d pgd20 cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep1_test_pgd20_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 89.062500 %
Accuracy on 6464 cifar-10 test images: 90.965347 %
Accuracy ATTA5-ep1 ViT on 10000 pgd20 cifar-10 test images: 91.070000 %


# ATTA Training Epoch 2


In [18]:
# ATTA generate attack ep2
i = 0
t0 = time.time()
for (inputs, labels) in cifar_adv:
  adv_images = atk_pgd_5(inputs, labels)
  cifar_adv[i] = (adv_images.cpu(), labels.cpu())
  if i % 100 == 0:
    print("Progress(%): ", 64*i/500)
  i += 1

time_gen_atk_2 = time.time() - t0
print("time lap(sec):", time_gen_atk_2)

# Store
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd5_8p_adv_itr2.pt'
!touch $outfile_name
torch.save(cifar_adv, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  12.8
Progress(%):  25.6
Progress(%):  38.4
Progress(%):  51.2
Progress(%):  64.0
Progress(%):  76.8
Progress(%):  89.6
time lap(sec): 2457.4898376464844
ATTA5_8p_adv_itr1_model.pt  pgd5_8p_adv_itr2.pt		pgd_5_adv_itr3_model.pt
ATTA_5_adv_itr1_model.pt    pgd_5_adv_itr1.json		pgd_5_adv_itr3.pt
ATTA_5_adv_itr2_model.pt    pgd_5_adv_itr1_model.pt	pgd_5_adv_itr4_model.pt
pgd10_8p_adv_test.pt	    pgd_5_adv_itr1.pt		pgd_5_adv_itr4.pt
pgd_10_adv_test.pt	    pgd_5_adv_itr1_v2_model.pt	pgd_5_adv_itr5_model.pt
pgd20_8p_adv_test.pt	    pgd_5_adv_itr1_v2.pt	pgd_5_adv_itr5.pt
pgd_20_adv_test.pt	    pgd_5_adv_itr2_model.pt	README.gdoc
pgd5_8p_adv_itr1.pt	    pgd_5_adv_itr2.pt


In [8]:
# Load
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'pgd5_8p_adv_itr2.pt'
cifar_adv = torch.load(outfile_name)
len(cifar_adv)

782

In [19]:
# ATTA train ep2
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep2_train_sec"] = t1 - t0

# Store model after itr2
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr2_model.pt'
!touch $outfile_name
torch.save(ViT_model, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

result_dict

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 0.000000 %
running loss:  7.169466018676758
Training accuracy of ViT on 9664 adv exs: 82.740066 %
running loss:  115.25743697583675
Training accuracy of ViT on 19264 adv exs: 90.381022 %
running loss:  142.59680522605777
Training accuracy of ViT on 28864 adv exs: 93.164496 %
running loss:  163.8698865659535
Training accuracy of ViT on 38464 adv exs: 94.667741 %
running loss:  183.74595912732184
Training accuracy of ViT on 48064 adv exs: 95.568409 %
running loss:  201.96931966766715
Training accuracy on 50000 pgd-5 cifar-10 adv images: 95.700000 %
ATTA5_8p_adv_itr1_model.pt  pgd5_8p_adv_itr2.pt		pgd_5_adv_itr3_model.pt
ATTA_5_adv_itr1_model.pt    pgd_5_adv_itr1.json		pgd_5_adv_itr3.pt
ATTA_5_adv_itr2_model.pt    pgd_5_adv_itr1_model.pt	pgd_5_adv_itr4_model.pt
pgd10_8p_adv_test.pt	    pgd_5_adv_itr1.pt		pgd_5_adv_itr4.pt
pgd_10_adv_test.pt	    pgd_5_adv_itr1_v2_model.pt	pgd_5_adv_itr5_model.pt
pgd20_8p_adv_test.pt	    pgd_5_adv_itr1_v2.pt	pgd_5_adv

{'ATTA_ep1_Orig_Acc': 93.21,
 'ViT_ATTA_5_ep1_test_pgd10_acc': 91.29,
 'ViT_ATTA_5_ep1_test_pgd20_acc': 91.07,
 'ViT_ATTA_5_ep1_train_acc': 93.79,
 'ViT_ATTA_5_ep1_train_sec': 663.3380136489868,
 'ViT_ATTA_5_ep2_train_acc': 95.7,
 'ViT_ATTA_5_ep2_train_sec': 663.5174517631531}

In [9]:
optimizer = optim.Adam(ViT_model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# ATTA train ep2 with lower learning rate
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep2_train_sec"] = t1 - t0

# Store model after itr2
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr2_model_v2.pt'
!touch $outfile_name
torch.save(ViT_model, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

result_dict

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 3.125000 %
running loss:  7.169466018676758
Training accuracy of ViT on 9664 adv exs: 83.567881 %
running loss:  103.10189383476973
Training accuracy of ViT on 19264 adv exs: 90.915698 %
running loss:  124.51980127766728
Training accuracy of ViT on 28864 adv exs: 93.625277 %
running loss:  140.5249695573002
Training accuracy of ViT on 38464 adv exs: 95.018719 %
running loss:  154.42406614683568
Training accuracy of ViT on 48064 adv exs: 95.874251 %
running loss:  166.3265758799389
Training accuracy on 50000 pgd-5 cifar-10 adv images: 96.004000 %
ATTA5_8p_adv_itr1_model.pt     pgd5_8p_adv_itr5.pt
ATTA5_8p_adv_itr2_model.pt     pgd_5_adv_itr1.json
ATTA5_8p_adv_itr2_model_v2.pt  pgd_5_adv_itr1_model.pt
ATTA5_8p_adv_itr3_model.pt     pgd_5_adv_itr1.pt
ATTA5_8p_adv_itr4_model.pt     pgd_5_adv_itr1_v2_model.pt
ATTA5_8p_adv_itr5_model.pt     pgd_5_adv_itr1_v2.pt
ATTA_5_adv_itr1_model.pt       pgd_5_adv_itr2_model.pt
ATTA_5_adv_itr2_model.pt       pgd_5_

{'ViT_ATTA_5_ep2_train_acc': 96.004,
 'ViT_ATTA_5_ep2_train_sec': 661.7895069122314}

In [9]:
optimizer = optim.Adam(ViT_model.parameters(), lr=0.0004)
criterion = nn.CrossEntropyLoss()

# ATTA train ep2 with HIGHER learning rate
t0 = time.time()

i = 0
correct = 0
total = 0
running_loss = 0.0
for (inputs, labels) in cifar_adv:
  # inputs & labels already on device
  # Adv training
  optimizer.zero_grad()
  inputs = inputs.to(device)
  labels = labels.to(device)

  # forward + backward + optimize
  outputs = ViT_model(inputs)
  loss = criterion(outputs, labels)
  loss.backward()
  optimizer.step()
  running_loss += loss.item()
  
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs.data, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  # print("Here", np.sum(predicted == labels))
  if i % 150 == 0:
    print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
    print("running loss: ", running_loss)
  i += 1

print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_train_acc"] = 100 * correct / total
t1 = time.time()
result_dict["ViT_ATTA_5_ep2_train_sec"] = t1 - t0

# Store model after itr2
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr2_model_v3.pt'
!touch $outfile_name
torch.save(ViT_model, outfile_name)
!ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

result_dict

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Training accuracy of ViT on 64 adv exs: 39.062500 %
running loss:  7.169466018676758
Training accuracy of ViT on 9664 adv exs: 79.956540 %
running loss:  128.4574111700058
Training accuracy of ViT on 19264 adv exs: 88.797757 %
running loss:  167.62339539825916
Training accuracy of ViT on 28864 adv exs: 91.920732 %
running loss:  202.5880583152175
Training accuracy of ViT on 38464 adv exs: 93.500416 %
running loss:  234.90400163829327
Training accuracy of ViT on 48064 adv exs: 94.536451 %
running loss:  263.85921609960496
Training accuracy on 50000 pgd-5 cifar-10 adv images: 94.706000 %
ATTA5_8p_adv_itr1_model.pt     pgd5_8p_adv_itr4.pt
ATTA5_8p_adv_itr2_model.pt     pgd5_8p_adv_itr5.pt
ATTA5_8p_adv_itr2_model_v2.pt  pgd_5_adv_itr1.json
ATTA5_8p_adv_itr2_model_v3.pt  pgd_5_adv_itr1_model.pt
ATTA5_8p_adv_itr3_model.pt     pgd_5_adv_itr1.pt
ATTA5_8p_adv_itr4_model.pt     pgd_5_adv_itr1_v2_model.pt
ATTA5_8p_adv_itr5_model.pt     pgd_5_adv_itr1_v2.pt
ATTA_5_adv_itr1_model.pt       pgd_5_adv

{'ViT_ATTA_5_ep2_train_acc': 94.706,
 'ViT_ATTA_5_ep2_train_sec': 1154.379698753357}

In [ ]:
# Load model
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr2_model.pt'
ViT_model = torch.load(outfile_name)

In [ ]:
# Load model v2
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
outfile_name = output_dir + 'ATTA5_8p_adv_itr2_model_v2.pt'
ViT_model = torch.load(outfile_name)

# Evaluate ATTA Epoch 2 Model


In [10]:
# Epoch 2 Model on Original test data
correct = 0
total = 0
with torch.no_grad():
  for i, (inputs, labels) in enumerate(cifar_testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 50 == 0):
      print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ATTA_ep2_Orig_Acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy of the network on 64 cifar-10 test images: 75.000000 %
Accuracy of the network on 3264 cifar-10 test images: 71.415441 %
Accuracy of the network on 6464 cifar-10 test images: 71.565594 %
Accuracy of the network on 9664 cifar-10 test images: 71.864652 %
Accuracy of the network on 10000 cifar-10 test images: 71.750000 %


In [13]:
# Evaluate model on pgd-10 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test10):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep2 ViT on %d pgd10 cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_test_pgd10_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 75.000000 %
Accuracy on 6464 cifar-10 test images: 67.636139 %
Accuracy ATTA5-ep2 ViT on 10000 pgd10 cifar-10 test images: 67.240000 %


In [14]:
# Evaluate model on pgd-20 test adv
correct = 0
total = 0
for i, (inputs, labels) in enumerate(cifar_adv_test20):
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = ViT_model(inputs)
  _, predicted = torch.max(outputs, 1)
  total += labels.size(0)
  correct += (predicted == labels).sum().item()
  if(i % 100 == 0):
    print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

print('Accuracy ATTA5-ep2 ViT on %d pgd20 cifar-10 test images: %f %%' % (
    total,
    100 * correct / total))
result_dict["ViT_ATTA_5_ep2_test_pgd20_acc"] = 100 * correct / total

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Accuracy on 64 cifar-10 test images: 73.437500 %
Accuracy on 6464 cifar-10 test images: 67.991955 %
Accuracy ATTA5-ep2 ViT on 10000 pgd20 cifar-10 test images: 67.810000 %


# ATTA Training Epoch 3 ~


In [25]:
output_dir = '../gdrive/MyDrive/EECS598-007_Group/ATTA_output/'
t0 = time.time()
for ep in range(3):
  print("##########    Epoch " + str(ep+3) + "    ##########")
  ts = time.time()
  i = 0
  for (inputs, labels) in cifar_adv:
    adv_images = atk_pgd_5(inputs, labels)
    cifar_adv[i] = (adv_images.cpu(), labels.cpu())
    if i % 100 == 0:
      print("Progress(%): ", 64*i/500)
    i += 1
  time_gen_atk_2 = time.time() - ts
  print("time lap(sec):", time_gen_atk_2)
  time_name = "ViT_ATTA_5_ep" + str(ep+3) + "_gen_atk_sec"
  result_dict[time_name] = time_gen_atk_2

  # Store cifar-10
  outfile_name = 'pgd5_8p_adv_itr' + str(ep+3) + '.pt'
  outfile_name = output_dir + outfile_name
  !touch $outfile_name
  torch.save(cifar_adv, outfile_name)
  !ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

  ts = time.time()
  i = 0
  correct = 0
  total = 0
  running_loss = 0.0
  for (inputs, labels) in cifar_adv:
    # inputs & labels already on device
    # Adv training
    optimizer.zero_grad()
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward + backward + optimize
    outputs = ViT_model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    # print("Here", np.sum(predicted == labels))
    if i % 150 == 0:
      print('Training accuracy of ViT on %d adv exs: %f %%' % (total, 100 * correct / total))
      print("running loss: ", running_loss)
    i += 1

  print('Training accuracy on %d pgd-5 cifar-10 adv images: %f %%' % (
      total,
      100 * correct / total))
  t1 = time.time()
  acc_name = "ViT_ATTA_5_ep" + str(ep+3) + "_train_acc"
  time_name = "ViT_ATTA_5_ep" + str(ep+3) + "_train_sec"
  result_dict[acc_name] = 100 * correct / total
  result_dict[time_name] = t1 - ts
  print("Training time: ", t1 - ts)

  # Store model after itr ep+3
  outfile_name = 'ATTA5_8p_adv_itr' + str(ep+3) + '_model.pt'
  outfile_name = output_dir + outfile_name
  !touch $outfile_name
  torch.save(ViT_model, outfile_name)
  !ls ../gdrive/MyDrive/EECS598-007_Group/ATTA_output/

  #################################################################
  #####################    EVALUATE MODEL    ######################
  #################################################################

  # Epoch 2 Model on Original test data
  correct = 0
  total = 0
  with torch.no_grad():
    for i, (inputs, labels) in enumerate(cifar_testloader):
      inputs, labels = inputs.to(device), labels.to(device)
      with torch.no_grad():
        outputs = ViT_model(inputs)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      if(i % 50 == 0):
        print('Accuracy of the network on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

  print('Accuracy of the network on %d cifar-10 test images: %f %%' % (
      total,
      100 * correct / total))
  orig_acc_name = "ATTA_ep" + str(ep+3) + "_Orig_Acc"
  result_dict[orig_acc_name] = 100 * correct / total

  # Evaluate model on pgd-10 test adv
  correct = 0
  total = 0
  for i, (inputs, labels) in enumerate(cifar_adv_test10):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 100 == 0):
      print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

  print('Accuracy ATTA5-ep2 ViT on %d pgd10 cifar-10 test images: %f %%' % (
      total,
      100 * correct / total))
  pgd10_acc_name = "ViT_ATTA_5_ep" + str(ep+3) + "_test_pgd10_acc"
  result_dict[pgd10_acc_name] = 100 * correct / total

  # Evaluate model on pgd-20 test adv
  correct = 0
  total = 0
  for i, (inputs, labels) in enumerate(cifar_adv_test20):
    inputs, labels = inputs.to(device), labels.to(device)
    with torch.no_grad():
      outputs = ViT_model(inputs)
    _, predicted = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    if(i % 100 == 0):
      print('Accuracy on %d cifar-10 test images: %f %%' % (total, 100 * correct / total))

  print('Accuracy ATTA5-ep2 ViT on %d pgd20 cifar-10 test images: %f %%' % (
      total,
      100 * correct / total))
  pgd20_acc_name = "ViT_ATTA_5_ep" + str(ep+3) + "_test_pgd20_acc"
  result_dict[pgd20_acc_name] = 100 * correct / total

t1 = time.time()
result_dict["ViT_3eps_ATTA_cifar10_sec"] = t1 - t0

##########    Epoch 3    ##########


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Progress(%):  0.0
Progress(%):  12.8
Progress(%):  25.6
Progress(%):  38.4
Progress(%):  51.2
Progress(%):  64.0
Progress(%):  76.8
Progress(%):  89.6
time lap(sec): 2453.5920791625977
ATTA5_8p_adv_itr1_model.pt  pgd5_8p_adv_itr2.pt		pgd_5_adv_itr3_model.pt
ATTA5_8p_adv_itr2_model.pt  pgd5_8p_adv_itr3.pt		pgd_5_adv_itr3.pt
ATTA_5_adv_itr1_model.pt    pgd_5_adv_itr1.json		pgd_5_adv_itr4_model.pt
ATTA_5_adv_itr2_model.pt    pgd_5_adv_itr1_model.pt	pgd_5_adv_itr4.pt
pgd10_8p_adv_test.pt	    pgd_5_adv_itr1.pt		pgd_5_adv_itr5_model.pt
pgd_10_adv_test.pt	    pgd_5_adv_itr1_v2_model.pt	pgd_5_adv_itr5.pt
pgd20_8p_adv_test.pt	    pgd_5_adv_itr1_v2.pt	README.gdoc
pgd_20_adv_test.pt	    pgd_5_adv_itr2_model.pt
pgd5_8p_adv_itr1.pt	    pgd_5_adv_itr2.pt
Training accuracy of ViT on 64 adv exs: 1.562500 %
running loss:  8.26323127746582
Training accuracy of ViT on 9664 adv exs: 84.644040 %
running loss:  130.8826773762703
Training accuracy of ViT on 19264 adv exs: 88.963870 %
running loss:  192.23786

In [26]:
result_dict

{'ATTA_ep1_Orig_Acc': 93.21,
 'ATTA_ep2_Orig_Acc': 89.22,
 'ATTA_ep3_Orig_Acc': 84.82,
 'ATTA_ep4_Orig_Acc': 72.5,
 'ATTA_ep5_Orig_Acc': 78.2,
 'ViT_3eps_ATTA_cifar10_sec': 9635.233304738998,
 'ViT_ATTA_5_ep1_test_pgd10_acc': 91.29,
 'ViT_ATTA_5_ep1_test_pgd20_acc': 91.07,
 'ViT_ATTA_5_ep1_train_acc': 93.79,
 'ViT_ATTA_5_ep1_train_sec': 663.3380136489868,
 'ViT_ATTA_5_ep2_test_pgd10_acc': 81.74,
 'ViT_ATTA_5_ep2_test_pgd20_acc': 82.49,
 'ViT_ATTA_5_ep2_train_acc': 95.7,
 'ViT_ATTA_5_ep2_train_sec': 663.5174517631531,
 'ViT_ATTA_5_ep3_gen_atk_sec': 2453.5920791625977,
 'ViT_ATTA_5_ep3_test_pgd10_acc': 82.36,
 'ViT_ATTA_5_ep3_test_pgd20_acc': 82.49,
 'ViT_ATTA_5_ep3_train_acc': 94.0,
 'ViT_ATTA_5_ep3_train_sec': 660.7246961593628,
 'ViT_ATTA_5_ep4_gen_atk_sec': 2446.890088558197,
 'ViT_ATTA_5_ep4_test_pgd10_acc': 69.49,
 'ViT_ATTA_5_ep4_test_pgd20_acc': 69.91,
 'ViT_ATTA_5_ep4_train_acc': 95.178,
 'ViT_ATTA_5_ep4_train_sec': 660.5210947990417,
 'ViT_ATTA_5_ep5_gen_atk_sec': 2446.29837441